In [1]:
import sklearn.metrics
import pandas as pd
import csv

In [ ]:
d = hickle.load('/home/eben/projects/rbc_cnn/data/April_tommy_non_overlap.hkl')
labels = []
images = []

for y, x in zip(d['y'], d['X']):
    labels.append(y)
    images.append(x)
labels = np.array(labels)
images = np.array(images)
len(labels)

In [2]:
%run -n cnn.py

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Graphics Device (CNMeM is disabled, cuDNN 4007)


In [ ]:
output_layer, samplers = setup('April_total_non_overlap')

In [3]:
def get_freq(dataset, train):
    print 'Loading', dataset
    # load in hickle dataset containing cell label dataset
    d = hickle.load('{}/{}.hkl'.format(DATA_DIR, dataset))

    GROUPS = {
        'NORMAL': 0,
        'Echinocyte': 1,
        'Dacrocyte': 2,
        'Schistocyte': 3,
        'Elliptocyte': 4,
        'Acanthocyte': 5,
        'Target cell': 6,
        'Stomatocyte': 7,
        'Spherocyte': 8,
        'Overlap': 9
    }
    LABELS = {v:k for k, v in GROUPS.items()}
    
    labels = []
    images = []
    for y, x in zip(d['y'], d['X']):
        labels.append(GROUPS[y])
        images.append(x)
    labels = np.array(labels)
    images = np.array(images)

    sss = sklearn.cross_validation.StratifiedShuffleSplit(
        labels,
        n_iter=1,
        test_size=0.2,
        random_state=RANDOM_SEED,
        )
    
    if train:
        ix, _ = tuple(sss)[0]
    else:
        _, ix = tuple(sss)[0]

    labels = labels[ix]
    images = images[ix]
    
    images = images / 255. - 0.5
    images = images.transpose(0, 3, 1, 2)
    
    for i in range(len(LABELS)):
        print '{} - {}: {}'.format(i, LABELS[i], (labels == i).sum())
        
    return images, labels

In [4]:
# output_layer is model, f_pred is applying model to 
output_layer, samplers = setup('September_1_total_non_overlap')
f_pred = get_predictor(output_layer)

Layer 0 type: <class 'lasagne.layers.input.InputLayer'> output: (32, 3, 70, 70)
Layer 1 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 68, 68)
Layer 2 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 66, 66)
Layer 3 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 64, 64)
Layer 4 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 32, 31, 31)
Layer 5 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 32, 31, 31)
Layer 6 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 29, 29)
Layer 7 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 27, 27)
Layer 8 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 25, 25)
Layer 9 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 64, 12, 12)
Layer 10 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 64, 12, 12)
Layer 11 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 128, 10, 10

In [ ]:
print 'Dataset stats:'
X, y = get_freq('April_tommy_non_overlap', train=False)

In [ ]:
print 'Dataset stats:'
X, y = get_freq('April_rick_non_overlap', train=True)

In [ ]:
print 'Dataset stats:'
X, y = get_freq('April_total_non_overlap', train=True)

In [ ]:
print 'Dataset stats:'
X, y = get_data('April_tommy_non_overlap', train=False)

model = './output/models/tommy epoch 224.pickle'
print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (p.argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

In [ ]:
print 'Dataset stats:'
# validation set
X, y = get_data('April_rick_non_overlap', train=False)

# training set/model
model = './output/models/tommy epoch 224.pickle'
print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (p.argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

In [ ]:
cm = sklearn.metrics.confusion_matrix(y, p.argmax(1))
cm.shape

In [ ]:
print 'Dataset stats: test set'
X, y = get_data('April_rick_non_overlap', train=False)

model = './output/models/rick epoch 157.pickle'
print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (p.argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

In [ ]:
print 'Dataset stats: test set'

# get validation set
X, y = get_data('April_tommy_non_overlap', train=False)

#training
model = './output/models/rick epoch 157.pickle'

print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (f_pred(X.astype('float32')).argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

In [ ]:
print 'Dataset stats: test set'

X, y = get_data('April_total_non_overlap', train=False)

model = './output/models/total epoch 184.pickle'


print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (f_pred(X.astype('float32')).argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

In [6]:
def get_data(dataset, train):
    print 'Loading', dataset
    # load in hickle dataset containing cell label dataset
    d = hickle.load('{}/{}.hkl'.format(DATA_DIR, dataset))

    GROUPS = {
        'NORMAL': 0,
        'Echinocyte': 1,
        'Dacrocyte': 2,
        'Schistocyte': 3,
        'Elliptocyte': 4,
        'Acanthocyte': 5,
        'Target cell': 6,
        'Stomatocyte': 7,
        'Spherocyte': 8,
        'Overlap': 9
    }
    LABELS = {v:k for k, v in GROUPS.items()}
    
    labels = []
    images = []
    pk = []
    
    # z = pk_value
    for y, x, z in zip(d['y'], d['X'], d['pk']):
        labels.append(GROUPS[y])
        images.append(x)
        pk.append(z)
    labels = np.array(labels)
    images = np.array(images)
    pk = np.array(pk)

    
    sss = sklearn.cross_validation.StratifiedShuffleSplit(
        labels,
        n_iter=1,
        test_size=0.2,
        random_state=RANDOM_SEED,
        )
    
    # create train set
    if train:
        ix, _ = tuple(sss)[0]
    # create validation set
    else:
        _, ix = tuple(sss)[0]

    labels = labels[ix]
    images = images[ix]
    pk = pk[ix]
    
    images = images / 255. - 0.5
    images = images.transpose(0, 3, 1, 2)
    
    for i in range(len(LABELS)):
        print '{} - {}: {}'.format(i, LABELS[i], (labels == i).sum())
        
    return images, labels, pk

In [8]:
output_layer, samplers = setup('August_25_total_non_overlap')
print '\n'

print """Network parameter settings:
Batch size: {}
Image width: {}
Epochs: {}
Batch size: {}
Initial learning rate: {}
Momentum: {}
""".format(BATCH_SIZE, IMAGE_W, NUM_EPOCHS, BATCH_SIZE, LEARNING_RATE, MOMENTUM)

print 'Dataset stats: Training set'
X, y = get_freq('August_25_total_non_overlap', train=True)
print '\n'

print 'Dataset stats: Validation set'
X, y, z = get_data('August_25_total_non_overlap', train=False)

model = './output/models/test epoch 140.pickle'

print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (f_pred(X.astype('float32')).argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

Layer 0 type: <class 'lasagne.layers.input.InputLayer'> output: (32, 3, 60, 60)
Layer 1 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 58, 58)
Layer 2 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 56, 56)
Layer 3 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 54, 54)
Layer 4 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 32, 26, 26)
Layer 5 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 32, 26, 26)
Layer 6 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 24, 24)
Layer 7 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 22, 22)
Layer 8 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 20, 20)
Layer 9 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 64, 10, 10)
Layer 10 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 64, 10, 10)
Layer 11 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 128, 8, 8)


In [7]:

print '\n'

print """Network parameter settings:
Batch size: {}
Image width: {}
Epochs: {}
Batch size: {}
Initial learning rate: {}
Momentum: {}
""".format(BATCH_SIZE, IMAGE_W, NUM_EPOCHS, BATCH_SIZE, LEARNING_RATE, MOMENTUM)

print 'Dataset stats: Training set'
X, y = get_freq('September_1_total_non_overlap', train=True)
print '\n'

print 'Dataset stats: Validation set'
X, y, z = get_data('September_1_total_non_overlap', train=False)

model = './output/models/september_1 epoch 175.pickle'

print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (f_pred(X.astype('float32')).argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))



Network parameter settings:
Batch size: 32
Image width: 70
Epochs: 250
Batch size: 32
Initial learning rate: 0.0001
Momentum: 0.9

Dataset stats: Training set
Loading September_1_total_non_overlap
0 - NORMAL: 809
1 - Echinocyte: 251
2 - Dacrocyte: 71
3 - Schistocyte: 607
4 - Elliptocyte: 73
5 - Acanthocyte: 133
6 - Target cell: 581
7 - Stomatocyte: 88
8 - Spherocyte: 192
9 - Overlap: 184


Dataset stats: Validation set
Loading September_1_total_non_overlap
0 - NORMAL: 203
1 - Echinocyte: 63
2 - Dacrocyte: 18
3 - Schistocyte: 152
4 - Elliptocyte: 18
5 - Acanthocyte: 33
6 - Target cell: 145
7 - Stomatocyte: 22
8 - Spherocyte: 48
9 - Overlap: 46

Testing model "september_1"
Loaded network parameters from ./output/models/september_1 epoch 175.pickle

Accuracy:  0.974598930481

Confusion matrix:
[[200   1   1   0   0   0   0   1   0   0]
 [  1  61   0   0   0   1   0   0   0   0]
 [  1   0  13   3   0   0   0   0   1   0]
 [  0   0   1 147   0   3   0   0   1   0]
 [  0   0   0   1  17   

In [9]:
GROUPS = {
        'NORMAL': 0,
        'Echinocyte': 1,
        'Dacrocyte': 2,
        'Schistocyte': 3,
        'Elliptocyte': 4,
        'Acanthocyte': 5,
        'Target cell': 6,
        'Stomatocyte': 7,
        'Spherocyte': 8,
        'Overlap': 9
    }

In [10]:
wrong_pk = z[(f_pred(X.astype('float32')).argmax(1) != y)]
wrong_label = p.argmax(1)[(f_pred(X.astype('float32')).argmax(1) != y)]
right_label = y[(f_pred(X.astype('float32')).argmax(1) != y)]

In [11]:
x = None
y= None
z = None
with open('wrong_cells.csv', 'w') as myfile:
    writer = csv.writer(myfile, dialect='excel')
    writer.writerow(["correct_x", "predicted_y", "pk_value"])
    wr = [x, y, z]
    for each in zip(right_label, wrong_label, wrong_pk):
        writer.writerow(each)
